# An Example of using `minimod`

For this notebook, we will first create the data needs to be used for the program. Then we will instantiate the class and make a report of the results using the `report` method

In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2
import sys
import pandas as pd

import minimod as mm


Using Python-MIP package version 1.6.8


In [4]:
## Data

# This is how the data was processed, assuming we use the Katie_VA_Benefits_and_Costs_1_8_2019.xlsx file.

## First we get a path to the raw data
excel_file = "data/raw/Katie_VA_Benefits_and_Costs_1_8_2019.xlsx"

# Get "vasoilold" and get discounted benefits for constraint

discount_costs = 1/(1 + 0.03)


vasoilold_constraint = (
    pd.read_excel(excel_file,
                  sheet_name = 'Benefits',
                  header = 2)
    .loc[lambda df : df['intervention'] == 'vasoilold']
    .set_index(['intervention', 'space'])
    .stack()
    .to_frame()
    .reset_index()
    .rename({
        'level_2' : 'time',
        0 : 'benefit'
    }, axis= 'columns')
    .assign(
        time_rank = lambda df: (df['time'].rank(numeric_only=True, method= 'dense') -1).astype(int) ,
        time_discount_costs = lambda df: discount_costs**df['time_rank'],
        discounted_benefits = lambda df: df['time_discount_costs']*df['benefit']
    )
    ['discounted_benefits'].sum()
    
)


In [51]:
df_benefit = (pd.read_excel(excel_file, 
                           sheet_name = 'Benefits',
                           header = 2)
            #   .loc[lambda df: df['intervention'] != 'vasoilold']
              .set_index(['intervention', 'space'])
              .stack()
              .to_frame()
              .reset_index()
              .rename({'level_2' : 'time',
                       0 : 'benefit'}, axis=1)
              .set_index(['intervention', 'space', 'time'])
              )

df_cost = (pd.read_excel(excel_file, 
                           sheet_name = 'Costs',
                           header = 2)
         #   .loc[lambda df: df['intervention'] != 'vasoilold']
              .set_index(['intervention', 'space'])
              .stack()
              .to_frame()
              .reset_index()
              .rename({'level_2' : 'time',
                       0 : 'costs'}, axis=1)
              .set_index(['intervention', 'space', 'time'])
              )

In [52]:
# Then we merge the cost and benefit data together.

df = (df_benefit
      .merge(df_cost, left_index=True, right_index=True)
      .reset_index()    
)


# Then we save the data. The finished data can be found in the `/examples/data` folder.

df.to_csv("data/processed/example1.csv")


In [18]:
df = pd.read_csv("data/processed/example1.csv")

## Running the model

Now we instantiate the model, and then run `fit` and get the report.

In [53]:
c = mm.CostSolver(data = df, minimum_benefit = vasoilold_constraint)

In [54]:
opt = c.fit()

Loading MIP Model with:
              Solver = CBC
              Method = MIN,
              
[Note]: Optimal Solution Found


In [11]:
c.opt_df.reset_index().to_csv("hello.csv")

In [55]:
c.report()

opt_df = c.opt_df



              Optimized Scenario with:
              
              Method: MIN
              Discount Factor on Costs: 1.0
              Discount Factor on Benefits: 0.970873786407767
              
+-----------------------------+
+------------------------------+


Total Costs and Coverage by Year
+------------------------------+


|   time |   opt_vals |      opt_benefit |        opt_costs |
|-------:|-----------:|-----------------:|-----------------:|
|      1 |          3 | 796827           | 599226           |
|      2 |          3 | 879986           | 644275           |
|      3 |          6 |      1.59095e+06 |      1.30343e+06 |
|      4 |         11 |      2.76946e+06 |      2.05228e+06 |
|      5 |          7 |      1.6732e+06  |      1.19193e+06 |
|      6 |          9 |      2.56e+06    |      1.84151e+06 |
|      7 |          9 |      2.33104e+06 |      1.60902e+06 |
|      8 |          8 |      2.15703e+06 |      1.44381e+06 |
|      9 |          7 |      2.04059e+06 |  